In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import cv2; print('opencv', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
opencv 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


In [2]:
sys.path.extend(['./models'])
print('sys.path', sys.path)

sys.path ['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', './models']


In [3]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [4]:
model_name = 'unet3d'
dataset_name = 'cube_medi_l32'
wtype = 'medi'
labels = ['32']
unique_id = int(time.time())

img_slice, img_rows, img_cols, channels = 64, 64, 64, 1
num_classes = 1

batch_size = 16
epochs = 999

model_path = '../output/densenet3d/1563816118/model-best.hd5'

In [5]:
from models.unet_3d import UNET

model = UNET(input_shape = (img_slice, img_rows, img_cols, channels),
             model_path = None,
             batch_normalization=True, 
             dropout_rate=0.2, 
             weight_decay=1e-4)

model.summary()                                                    

W0729 14:09:00.930981 139772175509312 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


backbone output: (?, 4, 4, 4, 258)
backbone out1: <tensorflow.python.keras.layers.merge.Multiply object at 0x7f1d74275940> (?, 8, 8, 8, 228)
backbone out2: <tensorflow.python.keras.layers.merge.Multiply object at 0x7f1df01f5160> (?, 16, 16, 16, 168)
backbone out3: <tensorflow.python.keras.layers.core.Activation object at 0x7f1e8001d0b8> (?, 32, 32, 32, 24)
up6: (?, 8, 8, 8, 258)
merge6: (?, 8, 8, 8, 486)
(?, 8, 8, 8, 168)
(?, 8, 8, 8, 168)
u7: (?, 16, 16, 16, 168)
merge7: (?, 16, 16, 16, 336)
(?, 16, 16, 16, 32)
(?, 16, 16, 16, 32)
u8: (?, 32, 32, 32, 32)
merge8: (?, 32, 32, 32, 56)
(?, 32, 32, 32, 16)
(?, 32, 32, 32, 16)
u9: (?, 64, 64, 64, 16)
merge9: (?, 64, 64, 64, 17)
(?, 64, 64, 64, 16)
(?, 64, 64, 64, 16)
Model: "unet3d"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 64,  0          

In [6]:
print(model_name, len([l.name for l in model.layers if ('conv' in str(type(l)) or 'dense' in str(type(l)))])+1)

unet3d 89


In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

model = load_model(f'./output/{model_name}/1564368659/' + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

# # set backbone not trainable
# for i, layer in enumerate(model.layers):
#     if not isinstance(layer, BatchNormalization) and i<=327:
#         layer.trainable = False

W0729 14:09:02.274011 139772175509312 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 14:09:02.276190 139772175509312 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
from generator import get_segment_file_list, split_dataset, SegmentSequence
dataset = []

for pos_file in get_segment_file_list(wtype=wtype):
    if pos_file.split('/')[-1].strip('.png').split('_')[-1].strip('l') in labels:
        dataset.append(pos_file)
        
print('filtered dataset:', len(dataset))

trainset, validset, _ = split_dataset(dataset, train_by_valid=0.05)

gen_train = SegmentSequence(trainset, wtype=wtype, batch_size=batch_size)
gen_valid = SegmentSequence(validset, wtype=wtype, batch_size=batch_size)

print(len(gen_train), len(gen_valid))

dataset length: 53408
filtered dataset: 11395
train, valid, test: 10825 570 0
y_valid: [0, 0, 0, 0, 570]
y_train: [0, 0, 0, 0, 10825]
677 36


In [9]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

output_dir = './output/{0}/{1}/'.format(model_name, unique_id)
print('output:', output_dir)

# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
# sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

checkpoint = ModelCheckpoint(filepath=output_dir + "weights-e{epoch:03d}-{loss:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             save_freq='epoch',
                             load_weights_on_restart=False)

checkpoint_best = ModelCheckpoint(filepath=output_dir + "model-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=False,
                                  mode='auto', 
                                  save_freq='epoch',
                                  load_weights_on_restart=False)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=1e-8, cooldown=0, min_lr=0)
tensorBoard = TensorBoard(log_dir=output_dir + 'logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
csvLogger = CSVLogger(filename=output_dir + "train_logs.csv", separator=',', append=False)

# model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           shuffle=True,
#           verbose=1,
#           validation_data=(X_valid, Y_valid),
#           callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
#           )

output: ./output/unet3d/1564409333/


In [ ]:
model.fit_generator(generator=gen_train,
                    steps_per_epoch=len(gen_train),
                    epochs=epochs, 
                    verbose=1, 
                    callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger], 
                    validation_data=gen_valid, 
                    validation_steps=len(gen_valid),
                    validation_freq=1, 
                    class_weight=None, 
                    max_queue_size=10, 
                    workers=1, 
                    use_multiprocessing=False, 
                    shuffle=True, 
                    initial_epoch=0)


Epoch 1/999
 47/677 [=>............................] - ETA: 17:09 - loss: 0.2833 - dice_coef: 0.7555